In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/cis/home/dpacker/my_documents/vote-counts


In [63]:
import pandas as pd
from pathlib import Path
import numpyro
import jax
from numpyro import distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive
from jax import random, numpy as jnp
import matplotlib.pyplot as plt
import seaborn as sns
import os
from numpyro.contrib.funsor import config_enumerate, infer_discrete
from src import preprocessing
from scipy.stats import gaussian_kde

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

path_data = Path("data")
path_raw_data = path_data / "raw_data"

In [39]:
def evaluate_kde(points: jnp.ndarray, value: float):
  kde = jax.scipy.stats.gaussian_kde(points)
  return kde(value)

def get_pivot_odds(p_samples, num_voters_array):
  return jax.vmap(evaluate_kde, in_axes=(0, None))(p_samples.T, 0.5)[:, 0] / (num_voters_array + 1)

In [3]:
raw_data = pd.read_csv(path_raw_data / "Local_2018.csv")
data = preprocessing.preprocess(raw_data)
elections_dict = preprocessing.get_elections_dict(data)

/tmp/ipykernel_1053851/3180180344.py:1: DtypeWarning: Columns (2,3,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(path_raw_data / "Local_2018.csv")


In [4]:
county_ids = list(elections_dict.keys())
example_county_id = county_ids[400]
example_county_elections = elections_dict[example_county_id]
top_two_elections = {}
for election_id, election in example_county_elections.items():
    top_two_elections[election_id] = election.sort_values(by = "votes", ascending=False).iloc[:2]

In [5]:
def model_basic(num_voters=None, votes=None):
    p = numpyro.sample("p", dist.Beta(4, 4))
    numpyro.sample(
        "observed_votes", dist.Binomial(total_count=num_voters, probs=p), obs=votes
    )

In [6]:
ex_election = top_two_elections["CITY COUNCIL - COLUMBIA"]
num_voters = ex_election["votes"].sum()
votes_0 = ex_election.iloc[0]["votes"]

In [7]:
# Start from this source of randomness. We will split keys for subsequent operations.
rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)

# Run NUTS.
kernel = NUTS(model_basic)
num_samples = 2_000
mcmc = MCMC(kernel, num_warmup=1000, num_samples=num_samples)
mcmc.run(
    rng_key_,
    num_voters = num_voters,
    votes = votes_0
)
mcmc.print_summary()
samples_basic = mcmc.get_samples()

sample: 100%|██████████| 3000/3000 [00:04<00:00, 654.43it/s, 1 steps of size 1.06e+00. acc. prob=0.93]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
         p      0.50      0.01      0.50      0.48      0.52    581.12      1.00

Number of divergences: 0


In [14]:
def model_basic_batched(num_voters_array=None, votes_array=None):
    with numpyro.plate("election", len(num_voters_array)) as ind:
        p = numpyro.sample("p", dist.Beta(4, 4))
        numpyro.sample(
            "observed_votes",
            dist.Binomial(total_count=num_voters_array[ind], probs=p),
            obs=votes_array[ind],
        )

In [20]:
num_voters_list = []
votes_list = []
for key, election in top_two_elections.items():
    num_voters_list.append(election["votes"].sum())
    votes_list.append(election.iloc[0]["votes"])
num_voters_array = jnp.array(num_voters_list)
votes_array = jnp.array(votes_list)

In [21]:
# Run NUTS.
kernel = NUTS(model_basic_batched)
num_samples = 2_000
mcmc = MCMC(kernel, num_warmup=1000, num_samples=num_samples)
mcmc.run(
    rng_key_,
    num_voters_array,
    votes_array,
)
mcmc.print_summary()
samples_basic_batched = mcmc.get_samples()

  0%|          | 0/3000 [00:00<?, ?it/s]

sample: 100%|██████████| 3000/3000 [00:07<00:00, 397.98it/s, 7 steps of size 6.65e-01. acc. prob=0.90] 


                mean       std    median      5.0%     95.0%     n_eff     r_hat
      p[0]      0.50      0.01      0.50      0.48      0.52   4005.03      1.00
      p[1]      0.52      0.01      0.52      0.50      0.54   3666.99      1.00
      p[2]      0.85      0.00      0.85      0.84      0.85   3819.35      1.00
      p[3]      0.55      0.01      0.55      0.54      0.56   4184.84      1.00
      p[4]      0.53      0.01      0.53      0.51      0.54   3717.68      1.00
      p[5]      0.53      0.02      0.53      0.50      0.55   3502.39      1.00
      p[6]      0.52      0.01      0.52      0.51      0.54   4772.17      1.00
      p[7]      0.59      0.01      0.59      0.58      0.60   4286.73      1.00
      p[8]      0.63      0.01      0.63      0.62      0.64   5020.87      1.00

Number of divergences: 0


In [40]:
get_pivot_odds(samples_basic_batched["p"], num_voters_array)

Array([0.01962846, 0.00489824, 0.        , 0.        , 0.00205135,
       0.00728431, 0.00260642, 0.        , 0.        ], dtype=float32)

In [68]:
def model_C2_batched(num_voters_array=None, votes_array=None):
    p_sigma = numpyro.sample("p_sigma", dist.Exponential(rate=1.0))
    p_mu = numpyro.sample("p_mu", dist.Beta(4, 4))
    with numpyro.plate("election", len(num_voters_array)) as ind:
        p = numpyro.sample("p", dist.BetaProportion(p_mu, p_sigma))
        votes = votes_array[ind] if votes_array is not None else None
        numpyro.sample(
            "observed_votes",
            dist.Binomial(total_count=num_voters_array[ind], probs=p),
            obs=votes,
        )

In [69]:
# Run NUTS.
kernel = NUTS(model_C2_batched)
num_samples = 2_000
mcmc = MCMC(kernel, num_warmup=1000, num_samples=num_samples)
mcmc.run(
    rng_key_,
    num_voters_array,
    votes_array,
)
mcmc.print_summary()
samples_C1_batched = mcmc.get_samples()

  0%|          | 0/3000 [00:00<?, ?it/s]

sample: 100%|██████████| 3000/3000 [00:09<00:00, 307.02it/s, 7 steps of size 6.33e-01. acc. prob=0.90]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
      p[0]      0.50      0.01      0.50      0.48      0.53   4805.86      1.00
      p[1]      0.52      0.01      0.52      0.50      0.54   4597.42      1.00
      p[2]      0.85      0.00      0.85      0.84      0.85   3599.81      1.00
      p[3]      0.55      0.01      0.55      0.54      0.56   4964.40      1.00
      p[4]      0.53      0.01      0.53      0.51      0.54   4311.79      1.00
      p[5]      0.53      0.02      0.53      0.50      0.56   3961.47      1.00
      p[6]      0.52      0.01      0.53      0.51      0.54   3585.15      1.00
      p[7]      0.59      0.01      0.59      0.58      0.60   3039.09      1.00
      p[8]      0.63      0.01      0.63      0.62      0.64   3236.37      1.00
      p_mu      0.56      0.07      0.56      0.45      0.67   5317.44      1.00
   p_sigma      4.56      1.81      4.29      1.69      7.18   3151.02      1.00

Number of divergences: 0


In [83]:
p_mu = samples_C1_batched["p_mu"]
p_sigma = samples_C1_batched["p_sigma"]
jax.scipy.stats.gaussian_kde(dist.BetaProportion(p_mu, p_sigma).sample(rng_key))(0.5)

Array([1.4045492], dtype=float32)